In [ ]:
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from hyperfine_parser import parse_hyperfine
from valgrind_parser import parse_valgrind
from wrk2_parser import parse_wrk2
from grapher import grapher, getStyle



In [ ]:

def grapher_paramless(results, ax=plt):

    # groups = groupByLabel(results)



    zipped = [(r['label'], r['parameters']['variation'], float(r['measurement']), float(r['stddev'])) for r in results]
    zipped = sorted(zipped, key=lambda x: x[2])
    labels = {}
    for (l, p, m, s) in zipped:
        # unzipped = list(zip(*zipped))
        style = getStyle(l)
        ax.errorbar([f"{l}[{p}]"], [m], [s], fmt='o', label=l if l not in labels else None, linewidth=2, capsize=6, **style)
        labels[l] = 1

    # handles, labels = ax.get_legend()
    # by_label = dict(zip(labels, handles))
    # ax.legend(by_label.values(), by_label.keys())

    # ax.legend()
    ax.set_xlabel("Benchmarks")
    plt.setp(ax.get_xticklabels(), ha="right", rotation=45)

    ax.set_ylabel(results[0]['unit'])


In [ ]:
res = []


files = [
'../hot_split_bench/output/bench-libseff.json',
'../hot_split_bench/output/bench-native.json',
]

for x in files:
    with open(x) as f:
        res += parse_hyperfine(f)

fig = plt.figure()
ax = fig.add_subplot(111)
grapher_paramless(res, ax)
ax.set_ylim(0, 0.15)
ax.legend()
# (ax, ax2) = fig.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [1, 2]})

# grapher_paramless(list(filter(lambda x: x['measurement'] > 0.5, res)), ax)


plt.show()


In [ ]:
res = []


files = [
'../state_bench/output/bench-libseff.json',
'../state_bench/output/bench-native.json',
'../state_bench/output/bench-cpp-effects.json',
'../state_bench/output/bench-cppcoro.json',
'../state_bench/output/bench-libhandler.json',
'../state_bench/output/bench-libmpeff.json',
'../state_bench/output/bench-libmprompt.json',
]

for x in files:
    with open(x) as f:
        res += parse_hyperfine(f)



fig = plt.figure()
# ax = fig.add_subplot(111)
(ax, ax2) = fig.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [1, 2]})

grapher_paramless(res, ax2)
grapher_paramless(list(filter(lambda x: x['measurement'] > 0.5, res)), ax)

# ax.set_yscale('log')
ax.set_ylim(4, 15)
ax2.set_ylim(0, 2)
ax2.legend()

ax.spines['bottom'].set_visible(False)
ax.set_xlabel(None)
ax2.spines['top'].set_visible(False)
ax.xaxis.tick_top()
ax.tick_params(labeltop=False)  # don't put tick labels at the top
# ax2.xaxis.tick_bottom()

d = .015  # how big to make the diagonal lines in axes coordinates
# arguments to pass to plot, just so we don't keep repeating them
kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
ax.plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
ax.plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
ax2.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal


plt.show()


In [ ]:
res = []
with open('../context_switch_bench/output/bench-libseff.json') as f:
    res += parse_hyperfine(f)
with open('../context_switch_bench/output/bench-cppcoro.json') as f:
    res += parse_hyperfine(f)
with open('../context_switch_bench/output/bench-libco.json') as f:
    res += parse_hyperfine(f)

fig = plt.figure()
ax = fig.add_subplot(111)

grapher(res, ax)

plt.show()

In [ ]:
res = []
with open('../memory_bench/output/bench-libseff.yaml') as f:
    res += parse_valgrind(f)
with open('../memory_bench/output/bench-cppcoro.yaml') as f:
    res += parse_valgrind(f)
with open('../memory_bench/output/bench-libco.yaml') as f:
    res += parse_valgrind(f)

fig = plt.figure()
ax = fig.add_subplot(111)

grapher(res, ax)
ax.set_ylabel("memory consumed (bytes)")

plt.show()

In [ ]:
labels = ['cohttp_eio', 'libscheff', 'nethttp_go', 'rust_hyper']
conns = [100, 1000, 5000, 10000, 50000]
threads = [8]
rps = [800000]

# THREADS	= 8
# CONNECTIONS = 100 1000 5000 10000 50000
# RPS =  800000



res = []

for l in labels:
    for c in conns:
        for t in threads:
            for r in rps:
                name = f'../http_server_bench/output/bench-{l}-{c}-{t}-{r}.txt'
                with open(name) as f:
                    res += parse_wrk2(f)

fig = plt.figure()
ax = fig.add_subplot(111)

grapher(res, ax, parameter_name='Connections')
# ax.set_xscale('linear')

plt.show()

In [ ]:
labels = ['libscheff', 'cohttp_eio', 'nethttp_go', 'rust_hyper']
conns = [1000]
threads = [1, 8, 16, 32]
rps = [2000000]

res = []

for l in labels:
    for c in conns:
        for t in threads:
            for r in rps:
                name = f'../http_server_bench/output/bench-{l}-{c}-{t}-{r}.txt'
                with open(name) as f:
                    res += parse_wrk2(f)

fig = plt.figure()
ax = fig.add_subplot(111)

grapher(res, ax, parameter_name='Number of threads')

plt.show()